# With statement

<font color='#bb9af7'>with as</font> statement is a powerful tool to manage external resources in your programs. Uses <i>context managers</i> to enter and exit these streams e.g files & communication.

## Syntax

<font color='#bb9af7'>with</font> statement before the <i>context manager</i>, and optionally the <font color='#bb9af7'>as</font> keyword to alias an object generated as <i>resource</i>.

In [ ]:
import dotenv
import os

FILENAME = 'with.csv'

dotenv.load_dotenv()
folder = os.getenv('TEMP_FOLDER')
path = os.path.join(folder, FILENAME)

with open(path, 'r') as file:
    first = file.readline(1)
    print(f"First row is: '{first}'")

## Example

As mentioned before, this statement is often used to create resources. These examples will open, read/write, close files & perform HTTP requests. 

### HTTP GET Requests

When using <code>aiohttp</code> module, each request is handled asynchronously & must use <font color='#bb9af7'>with</font> statement. Based on <a href='https://realpython.com/python-with-statement/'>Leodanis Pozo Ramos</a>' example.

In [ ]:
# py -m pip install aiofiles
# py -m pip install aiohttp
# py -m pip install tldextract
import asyncio
import os

import aiofiles
import aiohttp
import dotenv

from tldextract import extract

dotenv.load_dotenv()
folder = os.getenv('TEMP_FOLDER')

def parse_url(url: str):
    return extract(url).domain

async def get(url: str):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            print(f"{url}: status -> {response.status}")
            html = await response.text()
            print(f"{url}: type -> {html[:17].strip()}")
            return (parse_url(url), html)

async def write(path, doc):
    async with aiofiles.open(path, mode='w', encoding='utf-8') as f:
        await f.write(doc)

async def main():
    documents = await asyncio.gather(
        get("https://realpython.com"),
        get("https://pycoders.com"),
    )

    tasks = [ write(f'{folder}/{url}.html', doc) for url, doc in documents ]
    await asyncio.gather(*tasks)        

# Python: asyncio.run(main())
# Jupyter: await main()
await main()